<a href="https://colab.research.google.com/github/Meghraj-Webllisto/code/blob/main/nlp_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi -L

GPU 0: Tesla K80 (UUID: GPU-84d823c5-d4ca-0ae8-434c-5b797859cd1a)


In [2]:
# Let Download some helper function
! wget http://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2021-05-12 11:21:18--  http://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py [following]
--2021-05-12 11:21:18--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2021-05-12 11:21:19 (87.5 MB/s) - ‘helper_

In [3]:
# Import Series of helper functions for the notebook
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

In [6]:
# Download the dataset
!wget "https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip"

# unzip data
unzip_data("nlp_getting_started.zip")

--2021-05-12 11:34:02--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.125.128, 74.125.23.128, 74.125.203.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.125.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.006s  

2021-05-12 11:34:02 (91.4 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [8]:
# reading csv using pandas
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [9]:
# Shuffle training dataframe
train_df_shuffled = train_df.sample(frac=1, random_state=42) # shuflle with random state=42 for reproducebility
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [10]:
test_df.head()
# test doesn't have target column

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [13]:
# how many examples of each class
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

1 = real disaster tweet
0 = not a real disaster tweet

In [15]:
# How many smapels total?
print(f"Total training smapels: {len(train_df)}" )
print(f"total test smaples: {len(test_df)}")
print(f"Total Samples: {len(train_df) + len(test_df)}")

Total training smapels: 7613
total test smaples: 3263
Total Samples: 10876


In [16]:
# Let visualizer some training example
import random 
random_index = random.randint(0, len(train_df)-5)  # Creaete radom index not highter than total number of samples
for row in train_df_shuffled[["text","target"]][random_index:random_index+5].itertuples():
    _, text, target = row
    print(f"Target: {target}", "(real disaster)" if target >0 else "(not real disaster)")
    print(f"Text: \n{text}\n")
    print(f"---\n")

Target: 1 (real disaster)
Text: 
Islamic State group in Egypt threatens to kill Croat hostage http://t.co/VdgfXYX3bw

---

Target: 0 (not real disaster)
Text: 
@HomeworldGym @thisisperidot D: What? That's a tragedy. You have a wonderful nose

---

Target: 0 (not real disaster)
Text: 
Owner of Chicago-Area Gay Bar Admits to Arson Scheme http://t.co/UBFr1URAFc #LGBT | https://t.co/AlnV51d95x

---

Target: 1 (real disaster)
Text: 
@ScriptetteSar @katiecool447 btw the 30th is actually next year casualty began 6th September 1986 so 2016 marks 30 years

---

Target: 0 (not real disaster)
Text: 
@Barbi_Twins We need help-horses will die! Please RT &amp; sign petition! Take a stand &amp; be a voice for them! #gilbert23 https://t.co/e8dl1lNCVu

---



# split data into trian and validation sets

In [19]:
from sklearn.model_selection import train_test_split

train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size=0.1, # 10% percent only for validations
                                                                            random_state=42) # for reproducebility

In [20]:
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [21]:
# view some sampels 
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object), array([0, 

# Converting text into number because it need numbers to run

In [27]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

# Use the default TextVectorization
text_vectorizer = TextVectorization(max_tokens=None,
                                    standardize="lower_and_strip_punctuation",
                                    split="whitespace",
                                    ngrams=None,
                                    output_mode="int",
                                    output_sequence_length=None,
                                    pad_to_max_tokens=True)


In [28]:
# find the average number of tokens(words) in training tweets
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [29]:
# setup text vectorization variables
max_vocab_length = 10000
max_length =15

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)

In [30]:
text_vectorizer.adapt(train_sentences)

In [31]:
# Creating a sample sentence 
smaple_sentence = " There's  a flood in my area."
text_vectorizer([smaple_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 408,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [35]:
# unqiue workds in vocabulary
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5] # most common tolens ([unk] for unknown words)
bottom_5_words = words_in_vocab[-5:]
print(f"Numbner of words in vocab: {len(words_in_vocab)}")
print(f"Top 5 Most common words: {top_5_words}")
print(f"Bottom 5 Most common words: {bottom_5_words}")

Numbner of words in vocab: 10000
Top 5 Most common words: ['', '[UNK]', 'the', 'a', 'in']
Bottom 5 Most common words: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


# Creating an Embeding using Embedding layer

In [38]:
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length,
                             output_dim=128,
                             embeddings_initializer='uniform',
                             input_length=max_length)
embedding

In [40]:
# lets check one sentence in embedding
random_sentences =  random.choice(train_sentences)

print(f"original sentence: {random_sentences}")
sample_embedding_output = embedding(text_vectorizer([random_sentences]))
sample_embedding_output

original sentence: MH370: debris found on reunion island. ?? #sad #tragedy #innocent #crash #mh370


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-3.3124913e-02,  2.1401297e-02,  3.6092650e-02, ...,
         -2.5846316e-02,  2.0713579e-02, -1.0290243e-02],
        [ 4.8278298e-02,  1.4427748e-02, -3.8271021e-02, ...,
          6.8104267e-03, -1.9259501e-02, -4.7169160e-02],
        [-3.4528278e-02,  2.0778660e-02,  5.7685263e-03, ...,
          2.2970486e-02, -3.8845085e-02,  4.2608980e-02],
        ...,
        [-4.4712462e-02, -7.0362911e-03,  1.0587275e-05, ...,
          3.3841822e-02, -4.5480907e-02, -4.7658969e-02],
        [-4.4712462e-02, -7.0362911e-03,  1.0587275e-05, ...,
          3.3841822e-02, -4.5480907e-02, -4.7658969e-02],
        [-4.4712462e-02, -7.0362911e-03,  1.0587275e-05, ...,
          3.3841822e-02, -4.5480907e-02, -4.7658969e-02]]], dtype=float32)>

In [42]:
type(sample_embedding_output)

tensorflow.python.framework.ops.EagerTensor

In [59]:
sample_embedding_output[0][-2]

<tf.Tensor: shape=(128,), dtype=float32, numpy=
array([-4.47124615e-02, -7.03629106e-03,  1.05872750e-05,  3.83601449e-02,
        4.37856950e-02,  2.71155499e-02, -2.55576130e-02, -1.32821091e-02,
        2.24846713e-02,  1.29378177e-02,  1.99980177e-02, -1.00337341e-03,
        4.38654087e-02,  1.58280768e-02, -4.30818200e-02,  1.37804635e-02,
       -3.38762775e-02, -2.36516837e-02, -4.27488200e-02,  2.37836875e-02,
       -2.00839881e-02, -2.59476900e-03,  4.87668999e-02,  3.91772278e-02,
        4.64743115e-02, -4.25559767e-02,  1.63127817e-02,  2.82877944e-02,
       -2.05856562e-03,  1.31562464e-02,  1.52385347e-02,  1.89583935e-02,
       -4.75967638e-02, -5.23281097e-03, -1.47229433e-02, -4.30888645e-02,
       -4.49175760e-03,  4.59511168e-02,  4.01650183e-02, -1.14192478e-02,
        2.21886523e-02,  3.11229862e-02, -1.51832812e-02, -2.48932596e-02,
        2.11206935e-02, -4.37480472e-02,  2.22921036e-02, -1.93437692e-02,
        3.25409211e-02, -2.45898124e-02,  3.60284708

Embedding is ready , let build model if we can , there is no we just me MRJ

# THE NAIVE BASE model(baseline)
![](https://cdn.journaldev.com/wp-content/uploads/2020/10/NAIVE-BAYES-1.png)

In [60]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Creating tokenization and modelling pipeline
model_0 = Pipeline([
                    ("tfidf", TfidfVectorizer()),
                    ("clf", MultinomialNB()),
])

# Fit the pipeline to the training data
model_0.fit(train_sentences, train_labels)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

The benifit of shallow model like MultinomialNB is that trianing is very fast.

In [61]:
baseline_score = model_0.score(val_sentences, val_labels)
print(f"baseline model has an accuracy of: {baseline_score*100:.2f}%")

baseline model has an accuracy of: 79.27%


In [62]:
# makeing prediction
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])